In [144]:
import json
import pandas as pd

In [145]:
def format_number(x):
    if isinstance(x, float):
        if x.is_integer():
            return int(x)  # Convert to integer if the float is a whole number
        else:
            return '{:.2f}'.format(x).rstrip('0').rstrip('.')  # Keep up to 2 decimals, remove trailing zeros
    return x

def flatten_json(json_data):
    data = []
    for circuit, circuit_value in json_data.items(): # circuit size
        for benchmark, benchmark_value in circuit_value.items(): # benchmark_name
            for layout, metrics in benchmark_value.items():
                if layout == 'init':
                    continue
                if 'lookahead_swap' not in metrics:
                    continue
                basic_swap = metrics.get('basic_swap', None)
                sabre_swap = metrics.get('sabre_swap', None)
                lookahead_swap = metrics.get('lookahead_swap', None)
                swap_change = ((lookahead_swap - sabre_swap) / sabre_swap * 100) if sabre_swap != 0 and sabre_swap is not None and lookahead_swap is not None else None
                
                basic_depth = metrics.get('basic_depth', None)
                sabre_depth = metrics.get('sabre_depth', None)
                lookahead_depth = metrics.get('lookahead_depth', None)
                depth_change = ((lookahead_depth - sabre_depth) / sabre_depth * 100) if sabre_depth != 0 and sabre_depth is not None and lookahead_depth is not None else None

                data.append({
                    "benchmark": benchmark,
                    "size": circuit, # circuit size
                    "gate": benchmark_value['init']['size'], # total gates
                    "depth": benchmark_value['init']['depth'], # circuit depth
                    "layout": layout,
                    "swap basic": basic_swap, # basic swap
                    "swap sabre": sabre_swap, # sabre swap
                    "swap lookahead": lookahead_swap, # lookahead swap
                    "swap percentage": swap_change,
                    "depth basic": basic_depth, # basic depth
                    "depth sabre": sabre_depth, # sabre depth
                    "depth lookahead": lookahead_depth, # lookahead depth
                    "depth percentage": depth_change
                })
    return pd.DataFrame(data)


In [146]:
def rearrange_columns(df, first_col_name):
    """
    Reorder the columns of the DataFrame so that the specified column is first.
    
    Parameters:
    - df: The input DataFrame.
    - first_col_name: The name of the column to move to the first position.
    
    Returns:
    - A DataFrame with the specified column moved to the first position.
    """
    # Check if the specified column exists in the DataFrame
    if first_col_name not in df.columns:
        raise ValueError(f"Column '{first_col_name}' does not exist in the DataFrame.")
    
    # Create a list of columns with the specified column first
    columns = [first_col_name] + [col for col in df.columns if col != first_col_name]
    
    # Reorder the DataFrame
    df_reordered = df[columns]
    
    return df_reordered

In [147]:
def open_json_from_file():
    with open("result/benchmarking_2.json") as json_file:
        return json.load(json_file)

dict_benchmark = open_json_from_file()
print(len(dict_benchmark["5"].keys()))

15


In [148]:
a = dict_benchmark["5"]["ghz"]["full_10_2"]
a_sabre_swap = a.get("sabre_swap", None)
print(isinstance(a_sabre_swap, int))

True


In [149]:
df = flatten_json(dict_benchmark)
df = df.map(format_number)

def rename_columns(df):
    df.columns = [
        "benchmark",
        # "n",
        "g",
        "d",
        "layout",
        "s basic", # basic swap
        "s sabre", # sabre swap
        "s look", # lookahead swap
        "swap (%)",
        "d basic", # basic depth
        "d swap", # sabre depth
        "d look", # lookahead depth
        "d (%)",
    ]
    return df
# df = rename_columns(df)

print(df)

    benchmark size  gate  depth            layout swap basic swap sabre  \
0          dj    5    36     11         full_10_2          0          0   
1          dj    5    36     11          full_7_3          0          3   
2          dj    5    36     11         ring_10_2         36          3   
3          dj    5    36     11          ring_7_3         24          3   
4          dj    5    36     11          grid_9_3          9          3   
..        ...  ...   ...    ...               ...        ...        ...   
400    wstate   15   253    135  t_horizontal_5_4         63         21   
401    wstate   15   253    135    t_vertical_5_4        126         45   
402    wstate   15   253    135          ring_7_3          0         15   
403    wstate   15   253    135          full_7_3          0         12   
404    wstate   15   253    135          ring_5_4          0         48   

     swap lookahead swap percentage depth basic depth sabre  depth lookahead  \
0                 0

In [159]:
def save_latex_to_file(latex, circuit_size, filename=''):
    with open(f"table/table_{filename}_{circuit_size}.tex", "w") as file:
        file.write(latex)

def save_excel(excel, circuit_size, filename=''):
     with open(f"excel/excel_{filename}_{circuit_size}.tex", "w") as file:
        file.write(excel)

In [151]:
def build_by_benchmark(df, circuit_size):
    df = df[df['size'] == f"{circuit_size}"]
    # df_n_5 = df_n_5.replace('_', r'\_', regex=True)
    df = df.sort_values(by=['gate', 'benchmark'])
    df = df.drop(columns=['size'])
    df = rename_columns(df)
    latex = df.to_latex(index=False, escape=False, longtable=True, caption=f"Additional swap gates and circuit depth, n = {circuit_size}", label=f"benchmark-table-{circuit_size}")
    latex = latex.replace('_', r'\_').replace('%', r'\%')
    save_latex_to_file(latex, circuit_size, 'benchmark')
    return df

In [152]:
df_benchmark_5 = build_by_benchmark(df, 5)
df_benchmark_10 = build_by_benchmark(df, 10)
df_benchmark_15 = build_by_benchmark(df, 15)

In [153]:
def build_by_layout(df, circuit_size):
    df = df[df['size'] == f"{circuit_size}"]
    df = df.groupby(by=['layout']).apply(lambda x: x.sort_values(by=['gate', 'benchmark']))
    df = df.drop(columns=['size']) # remove circuit size
    df = rename_columns(df)
    df = rearrange_columns(df, 'layout')
    latex = df.to_latex(index=False, escape=False, longtable=True, caption=f"Additional swap gates and circuit depth, n = {circuit_size}", label=f"benchmark-table-{circuit_size}")
    latex = latex.replace('_', r'\_').replace('%', r'\%')
    save_latex_to_file(latex, circuit_size, 'layout')
    return df

In [160]:
def build_by_layout_excel(df, circuit_size):
    df = df[df['size'] == f"{circuit_size}"]
    df = df.groupby(by=['layout']).apply(lambda x: x.sort_values(by=['gate', 'benchmark']))
    # df = df.drop(columns=['size']) # remove circuit size
    # df = rename_columns(df)
    df = rearrange_columns(df, 'layout')
    excel = df.to_excel(excel_writer=f"excel/excel_layout_{circuit_size}", index=True, header=True)
    # latex = latex.replace('_', r'\_').replace('%', r'\%')
    # save_latex_to_file(excel, circuit_size, 'layout')
    return df

In [154]:
df_layout_5 = build_by_layout(df, 5)
df_layout_10 = build_by_layout(df, 10)
df_layout_15 = build_by_layout(df, 15)

/tmp/ipykernel_16169/3027913658.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(by=['layout']).apply(lambda x: x.sort_values(by=['gate', 'benchmark']))
/tmp/ipykernel_16169/3027913658.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(by=['layout']).apply(lambda x: x.sort_values(by=['gate', 'benchmark']))
/tmp/ipykernel_16169/3027913658.py:3: DeprecationWarning: DataFram

In [161]:
df_excel_layout_5 = build_by_layout_excel(df, 5)

/tmp/ipykernel_16169/1947387839.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(by=['layout']).apply(lambda x: x.sort_values(by=['gate', 'benchmark']))


TypeError: NDFrame.to_excel() missing 1 required positional argument: 'excel_writer'

In [155]:
print("Benchmark [5]\n", df_benchmark_5.value_counts("benchmark"))
print("Benchmark [10]\n", df_benchmark_10.value_counts("benchmark"))
print("Benchmark [15]\n", df_benchmark_15.value_counts("benchmark"))

Benchmark [5]
 benchmark
dj                10
ghz               10
graphstate        10
qaoa              10
qft               10
qftentangled      10
qnn               10
vqe               10
wstate            10
portfolioqaoa      9
portfoliovqe       9
random             9
realamprandom      9
su2random          9
twolocalrandom     9
Name: count, dtype: int64
Benchmark [10]
 benchmark
dj                10
ghz               10
graphstate        10
qaoa              10
vqe               10
wstate            10
qft                9
qftentangled       9
portfolioqaoa      8
portfoliovqe       8
qnn                8
random             8
realamprandom      8
su2random          8
twolocalrandom     8
Name: count, dtype: int64
Benchmark [15]
 benchmark
dj                10
ghz               10
graphstate        10
qaoa              10
vqe               10
wstate            10
qft                8
qftentangled       8
realamprandom      8
twolocalrandom     8
portfolioqaoa      7
portfoliov

In [158]:
df_layout_15

layout      benchmark     g    d s basic s sabre  \
layout                                                                         
full_10_2      288       full_10_2            ghz    17   17       0       6   
               278       full_10_2             dj   118   22      66       9   
               298       full_10_2     graphstate   150   29      30       6   
               385       full_10_2            vqe   253   31       0       6   
               395       full_10_2         wstate   253  135       0      12   
...                            ...            ...   ...  ...     ...     ...   
t_vertical_5_4 376  t_vertical_5_4      su2random   675   81    5304    1086   
               354  t_vertical_5_4            qnn   914  158    3669     600   
               314  t_vertical_5_4  portfolioqaoa  1260  192    5304     879   
               361  t_vertical_5_4         random  1992  412    2475    1203   
               321  t_vertical_5_4   portfoliovqe  2505  327    5304     942   

                    s look swap (%) d basic d swap  d look   d (%)  
layout                                                              
full_10_2      288       0     -100      17     20      17     -15  
               278       9        0      95     33      29  -12.12  
               298      24      300      51     40      34     -15  
               385       0     -100      31     41      31  -24.39  
               395       0     -100     135    141     135   -4.26  
...                    ...      ...     ...    ...     ...     ...  
t_vertical_5_4 376    1098      1.1    1962    658     265  -59.73  
               354    1077     79.5    1449    509     344  -32.42  
               314    1440    63.82    2150    641     430  -32.92  
               361    1800    49.63    2366   1214     658   -45.8  
               321    1251     32.8    2280    834     456  -45.32  

[127 rows x 12 columns]